In [20]:
from google.colab import drive
drive.mount('/content/drive')  # Authorize Drive access


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os

# Define dataset path in Google Drive
base_path = "/content/drive/MyDrive/COVID19/"
frame_path = os.path.join(base_path, "frames")  # CT scan images
mask_path = os.path.join(base_path, "masks")    # Segmentation masks

# Check if dataset folders exist
if not os.path.exists(frame_path) or not os.path.exists(mask_path):
    raise FileNotFoundError("Error: Dataset folders are missing!")

print(f" Total CT scan images: {len(os.listdir(frame_path))}")
print(f" Total segmentation masks: {len(os.listdir(mask_path))}")


 Total CT scan images: 2729
 Total segmentation masks: 2729


In [22]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

IMG_SIZE = (256, 256)

# Load images and masks
def load_dataset(image_folder, mask_folder):
    images, masks = [], []
    mask_files = set(os.listdir(mask_folder))  # Ensure mask exists

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        mask_path = os.path.join(mask_folder, filename)

        if filename in mask_files:
            img = load_img(img_path, target_size=IMG_SIZE, color_mode="grayscale")
            img = img_to_array(img) / 255.0  # Normalize

            mask = load_img(mask_path, target_size=IMG_SIZE, color_mode="grayscale")
            mask = img_to_array(mask) / 255.0  # Normalize

            images.append(img)
            masks.append(mask)
        else:
            print(f" Warning: Mask missing for {filename}")

    return np.array(images), np.array(masks)

train_images, train_masks = load_dataset(frame_path, mask_path)
print(f" Loaded {train_images.shape[0]} images and {train_masks.shape[0]} masks.")


 Loaded 2729 images and 2729 masks.


In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model

def unet_resnet(input_size=(256, 256, 1)):
    inputs = Input(input_size)

    # Encoder (ResNet-like feature extractor)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Decoder (U-Net upsampling path)
    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = Concatenate()([conv3, up5])
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    merge6 = Concatenate()([conv2, up6])
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    merge7 = Concatenate()([conv1, up7])
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge7)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [24]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define paths
base_path = "/content/drive/MyDrive/COVID19/"
frame_path = os.path.join(base_path, "frames")  # CT scan images
mask_path = os.path.join(base_path, "masks")    # Segmentation masks

IMG_SIZE = (256, 256)

def load_dataset(image_folder, mask_folder):
    images, masks = [], []
    mask_files = set(os.listdir(mask_folder))  # Ensure mask exists

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        mask_path = os.path.join(mask_folder, filename)

        if filename in mask_files:
            img = load_img(img_path, target_size=IMG_SIZE, color_mode="grayscale")
            img = img_to_array(img) / 255.0  # Normalize
            img = np.expand_dims(img, axis=-1)  # Ensure shape (256,256,1)

            mask = load_img(mask_path, target_size=IMG_SIZE, color_mode="grayscale")
            mask = img_to_array(mask) / 255.0  # Normalize
            mask = np.expand_dims(mask, axis=-1)  # Ensure shape (256,256,1)

            images.append(img)
            masks.append(mask)

    return np.array(images), np.array(masks)

# Load images and masks
train_images, train_masks = load_dataset(frame_path, mask_path)

# Save for later use
np.save("/content/drive/MyDrive/COVID19/train_images.npy", train_images)
np.save("/content/drive/MyDrive/COVID19/train_masks.npy", train_masks)

print(f" Data saved! Images: {train_images.shape}, Masks: {train_masks.shape}")


 Data saved! Images: (2729, 256, 256, 1, 1), Masks: (2729, 256, 256, 1, 1)


In [25]:
model = unet_resnet()
model.fit(train_images, train_masks, epochs=10, batch_size=8, validation_split=0.2)

# Save model in Google Drive
model_path = "/content/drive/MyDrive/COVID19_Dataset/model_weights.h5"
model.save(model_path)
print("Model trained and saved successfully!")


Epoch 1/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 102s 343ms/step - accuracy: 0.9647 - loss: 0.1120 - val_accuracy: 0.9835 - val_loss: 0.0436
Epoch 2/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 129s 314ms/step - accuracy: 0.9857 - loss: 0.0563 - val_accuracy: 0.9888 - val_loss: 0.0355
Epoch 3/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 142s 313ms/step - accuracy: 0.9863 - loss: 0.0437 - val_accuracy: 0.9900 - val_loss: 0.0334
Epoch 4/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 142s 313ms/step - accuracy: 0.9893 - loss: 0.0302 - val_accuracy: 0.9841 - val_loss: 0.0773
Epoch 5/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 146s 326ms/step - accuracy: 0.9919 - loss: 0.0213 - val_accuracy: 0.9916 - val_loss: 0.0259
Epoch 6/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 138s 312ms/step - accuracy: 0.9929 - loss: 0.0186 - val_accuracy: 0.9912 - val_loss: 0.0320
Epoch 7/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 89s 325ms/step - accuracy: 0.9933 - loss: 0.0167 - val_accuracy: 0.9904 - val_loss: 0.0365
Epoch 8/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 139s 312ms/step - accuracy: 0.9936 - 

Model trained and saved successfully!


In [26]:
!pip install fastapi uvicorn python-multipart


In [27]:
from fastapi import FastAPI, UploadFile, File
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
from io import BytesIO

app = FastAPI()

# Load trained model
model_path = "/content/drive/MyDrive/COVID19_Dataset/model_weights.h5"
model = tf.keras.models.load_model(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = load_img(BytesIO(await file.read()), target_size=(256, 256), color_mode="grayscale")
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image)[0]
    segmented = (prediction > 0.5).astype(np.uint8) * 255

    _, buffer = cv2.imencode(".png", segmented)
    return {"segmented_image": buffer.tobytes()}


In [29]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [30]:
import streamlit as st
import requests
from PIL import Image
import io


In [42]:
import streamlit as st
import requests
from PIL import Image
import io

st.title("COVID-19")

uploaded_file = st.file_uploader("Upload a CT scan image", type=["png", "jpg", "jpeg"])

if uploaded_file:
    files = {"file": uploaded_file.getvalue()}
    response = requests.post("http://127.0.0.1:8000/predict/", files=files)

    if response.status_code == 200:
        image = Image.open(io.BytesIO(response.content))
        st.image(image, caption="Segmented Output")
    else:
        st.error("Prediction failed!")


2025-03-12 03:05:02.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-12 03:05:02.732 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-12 03:05:02.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-12 03:05:02.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-12 03:05:02.734 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-12 03:05:02.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-12 03:05:02.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [32]:
!pip install streamlit pyngrok


In [33]:
!pip install streamlit pyngrok


In [34]:
%%writefile app.py
import streamlit as st

st.title("COVID-19 Segmentation App")
st.write("Upload a CT scan image to segment COVID-19 affected areas.")


Writing app.py
